In [2]:
import pandas as pd
import numpy as np
import re

import urllib 
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import json
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #or “last_expr”
from datetime import date
import calendar
from datetime import time, timedelta


# Getting Data

In [3]:
a = pd.read_parquet('/Users/aidanflood14/Downloads/20190807_161525_00140_c4sx9_3ac07b18-5095-4b99-8dfc-e32ecb92f085')
b = pd.read_parquet('/Users/aidanflood14/Downloads/20190807_161525_00140_c4sx9_17d59fa2-4e3e-448d-9e39-09ffc265160f')
c = pd.read_parquet('/Users/aidanflood14/Downloads/20190807_161525_00140_c4sx9_6565db64-53e0-4cfd-bf2d-9cc8590e8dee')
d = pd.read_parquet('/Users/aidanflood14/Downloads/20190805_201513_00239_c4sx9_cec68909-6cc6-4671-b42b-d1ac07394600')
e = pd.read_parquet('/Users/aidanflood14/Downloads/20190805_201513_00239_c4sx9_eb085114-e37b-49c0-baea-22e95a4371e7')
f = pd.read_parquet('/Users/aidanflood14/Downloads/20190805_201513_00239_c4sx9_532358aa-de1e-4ca7-83c2-aa0f6108fee9')

df = pd.concat([a,b,c,d,e,f]).sort_values('location_at').reset_index(drop = True)
df['location_at'] = df['location_at'].apply(lambda x: x - timedelta(hours = 5))
df['day_of_week'] = df['location_at'].apply(lambda x: calendar.day_name[x.weekday()])
df = df[df['day_of_week'].isin(['Thursday', 'Friday', 'Saturday', 'Sunday'])].reset_index(drop=True)
df['time'] = df['location_at'].apply(lambda x: x.time())

In [4]:
df.to_csv('/Users/aidanflood14/Desktop/X-Mode/LollapaloozaData.csv')

---

# Messing Around with Kepler

In [112]:
import keplergl
from keplergl import KeplerGl
map_1 = KeplerGl()

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


In [114]:
map_1.add_data(data=df[df['advertiser_id'].isin(weird)])


# Amount of People Each Day

In [5]:
thurs_total = len(df[df['day_of_week'] == 'Thursday'].advertiser_id.unique())
fri_total = len(df[df['day_of_week'] == 'Friday'].advertiser_id.unique())
sat_total = len(df[df['day_of_week'] == 'Saturday'].advertiser_id.unique())
sun_total = len(df[df['day_of_week'] == 'Sunday'].advertiser_id.unique())
total_dict = {'Friday' : fri_total, 'Thursday' : thurs_total, 'Saturday':sat_total, 'Sunday':sun_total}
total_dict

{'Friday': 7827, 'Thursday': 6497, 'Saturday': 7236, 'Sunday': 7673}

In [10]:
def amount_of_people():
    while True:
        try:
            day_options = ['Thursday', 'Friday', 'Saturday', 'Sunday']
            for i in range(len(day_options)):
                print(str(i+1) + ":", day_options[i])

                # Take user input and get the corresponding item from the list
            day_selected = int(input("Enter a number: "))
            day_selected = day_options[day_selected - 1]
            print("\n")
            break
        except:
            print("\n")
            print("Error: You must enter a valid number")
            print("\n")
    
    print('You selected ' + day_selected)
    print("\n")
    while True:
        try:
            stage_options = ['T Mobile Stage', 'Bud Light Stage', 'Lake Shore Stage', "Tito's Stage", 'BMI Stage', "Perry's Stage", 'American Eagle Stage', 'Kidzapalooza']
            # Print out your options
            for i in range(len(stage_options)):
                print(str(i+1) + ":", stage_options[i])

                # Take user input and get the corresponding item from the list
            stage = int(input("Enter a number: "))
            stage = stage_options[stage - 1]
            print("\n")
            break
        except:
            print("\n")
            print("Error: You must enter a valid number")
            print("\n")
    print('You selected ' + stage)
    print("\n")
    while True:
        try:
            start_time= input('Start time in military time: ').split(":")
            start_time_hour = int(start_time[0])
            start_time_minute = int(start_time[1])
            start_time_valid = time(start_time_hour, start_time_minute)
            break
        except:
            print("\n")
            print("Enter data in this format 'hh:mm' with hour in 0 to 23 and minutes in 0 to 59")
            print("\n")
              
        
    while True:
        try:
            end_time = input('End time in military time: ').split(":")
            end_time_hour = int(end_time[0])
            end_time_minute = int(end_time[1])
            end_time_valid = time(end_time_hour, end_time_minute)
            break
        except:
            print("\n")
            print("Enter data in this format 'hh:mm' with hour in 0 to 23 and minutes in 0 to 59")
            print("\n")
            
    time_range = [start_time_valid, end_time_valid]
    
    
    temp_df = df[(df['geo_name'] == stage)& (df['day_of_week'] == day_selected)&(df['time'] >= time_range[0]) & (df['time'] 
        <= time_range[1])].drop_duplicates('advertiser_id').groupby(['geo_name','day_of_week']).count().reset_index()
    temp_df['Total People at Park that Day'] = temp_df['day_of_week'].map(total_dict)
    temp_df['Percentage Represented of People that Day'] = 100 * np.array(temp_df['advertiser_id']) / np.array(temp_df['Total People at Park that Day'])
    temp_df.rename(columns = {'advertiser_id':'People Attending Stage at that Time', 'geo_name': 'Stage Selected', 'day_of_week': 'Day Selected'}, inplace = True)
    temp_df = temp_df.drop(['latitude', 'longitude', 'location_at', 'horizontal_accuracy', 'speed', 'geo_description', 'time'], axis = 1).sort_values('Total People at Park that Day').reset_index(drop = True)
    
    return temp_df
    
    

In [11]:
amount_of_people()


1: Thursday
2: Friday
3: Saturday
4: Sunday
Enter a number: ndh


Error: You must enter a valid number


1: Thursday
2: Friday
3: Saturday
4: Sunday
Enter a number: select * from


Error: You must enter a valid number


1: Thursday
2: Friday
3: Saturday
4: Sunday
Enter a number: 3


You selected Saturday


1: T Mobile Stage
2: Bud Light Stage
3: Lake Shore Stage
4: Tito's Stage
5: BMI Stage
6: Perry's Stage
7: American Eagle Stage
8: Kidzapalooza
Enter a number: 00


You selected Kidzapalooza


Start time in military time: fgn


Enter data in this format 'hh:mm' with hour in 0 to 23 and minutes in 0 to 59


Start time in military time: 9;00


Enter data in this format 'hh:mm' with hour in 0 to 23 and minutes in 0 to 59


Start time in military time: 0:20
End time in military time: 19:23


,Stage Selected,Day Selected,People Attending Stage at that Time,Total People at Park that Day,Percentage Represented of People that Day
0,Kidzapalooza,Saturday,156,7236,2.155887


In [ ]:
thurs_df = df[df['day_of_week'] == 'Thursday']
fri_df = df[df['day_of_week'] == 'Friday']
sat_df = df[df['day_of_week'] == 'Saturday']
sun_df = df[df['day_of_week'] == 'Sunday']

In [12]:
def amount_of_people(d):
    full_df = pd.DataFrame()
    days = list(d.keys())
    for day in days:
        day_dict = d.get(day)
        stages = list(day_dict.keys())
        for stage in stages:
            hours = day_dict.get(stage)
            for hour in hours:
                print('i')
                time_range = [hour[0],hour[1]]
                print(time_range)
                temp_df = df[(df['day_of_week'] == day) & (df['geo_name'] == stage)& (df['time'] > 
                    time_range[0]) & (df['time'] < time_range[1])].drop_duplicates('advertiser_id').groupby(['geo_name','day_of_week']).count().reset_index()
                temp_df['Total People at Park that Day'] = temp_df['day_of_week'].map(total_dict)
                temp_df['Percentage of People of Park'] = 100 * np.array(temp_df['advertiser_id']) / np.array(temp_df['Total People at Park that Day'])
                temp_df.rename(columns = {'advertiser_id':'People Attending Stage at that Time'}, inplace = True)
                temp_df = temp_df.drop(['latitude', 'longitude', 'location_at', 'horizontal_accuracy', 'speed', 'geo_description', 'time'], axis = 1).sort_values('Total People at Park that Day').reset_index(drop = True)
                full_df = pd.concat([full_df, temp_df])
    return full_df
    
    

In [13]:
amount_of_people(d)

i


IndexError: string index out of range